# Recommender Systems 2022/23

### Practice - AsySVD implemented with Python

AsymmetricSVD is a model-based matrix factorization algorithm in which the user latent factors are represented as a function of their user profile and of a second item factor matrix.

In [1]:
import time
import numpy as np

In [2]:
import scipy.sparse as sps

from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from challenge.utils.functions import read_data, evaluate_algorithm, generate_submission_csv


data_file_path = '../challenge/input_files/data_train.csv'
users_file_path = '../challenge/input_files/data_target_users_test.csv'
URM_all_dataframe, users_list = read_data(data_file_path, users_file_path)

URM_all = sps.coo_matrix(
    (URM_all_dataframe['Data'].values, (URM_all_dataframe['UserID'].values, URM_all_dataframe['ItemID'].values)))
URM_all = URM_all.tocsr()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)

In [3]:
URM_train

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>

### What do we need for AsySVD?

* Loss function
* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience


In [4]:
n_users, n_items = URM_train.shape

#### The two methods are based on two latent factor matrices $ W, V \in R^{I \times E}$ with E the embedding size, and biases

#### How to compute the predictions
$ \hat{r}_{ui} = \sum_{k=0}^{E}\sum_{j=0}^{I} r_{uj}W_{jk}H_{ki}$


#### The loss function we are interested in minimizing is
$L = |R - RWH|_F + \alpha|W|_2 + \beta|H|_2$

#### Gradients

$\frac{\partial}{\partial W} L = -2(R - RWH)RH + 2\alpha W $

$\frac{\partial}{\partial H} L = -2(R - RWH)RW + 2\alpha H $


#### The update is going to be (we can remove the coefficients)
$ W = W - \frac{\partial}{\partial W}$, or 

$ W = W + l((R - RWH)RH - \alpha W)$, with $l$ the learning rate


## Step 1: We create the dense latent factor matrices
### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [5]:
num_factors = 10

USER_profile_factors = np.random.random((n_items, num_factors))
ITEM_factors = np.random.random((n_items, num_factors))

In [6]:
USER_profile_factors

array([[0.18856831, 0.90217612, 0.98982947, ..., 0.94541454, 0.90624135,
        0.31908818],
       [0.44137743, 0.74073275, 0.44326409, ..., 0.40621576, 0.67268469,
        0.72549581],
       [0.8739436 , 0.98006013, 0.12750054, ..., 0.37312061, 0.47513745,
        0.33573713],
       ...,
       [0.42989664, 0.20879702, 0.45678952, ..., 0.34100331, 0.56038007,
        0.26818248],
       [0.65313656, 0.61938755, 0.90260646, ..., 0.19837236, 0.48963003,
        0.45819722],
       [0.26228609, 0.52205494, 0.70232682, ..., 0.2541192 , 0.04137471,
        0.38909485]])

In [7]:
ITEM_factors

array([[0.17843541, 0.35820796, 0.6182482 , ..., 0.15115074, 0.17562078,
        0.12034899],
       [0.15222427, 0.96352417, 0.44824266, ..., 0.10396651, 0.91391648,
        0.23865359],
       [0.96628695, 0.26157372, 0.58548373, ..., 0.52854501, 0.88103944,
        0.33321637],
       ...,
       [0.97229864, 0.7627337 , 0.31020948, ..., 0.23492599, 0.71148478,
        0.07988665],
       [0.54613659, 0.98855058, 0.36978015, ..., 0.82541137, 0.97344637,
        0.82363698],
       [0.36939083, 0.51889602, 0.54974387, ..., 0.40351463, 0.74113613,
        0.27692855]])

## Step 2: We sample an interaction and compute the prediction of the current model

In [8]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

268621

In [9]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

user_profile = URM_train[user_id]

(user_id, item_id, rating)

(9009, 101, 1.0)

In [10]:
# The estimated user factors may be divided by the square root of the profile length or the length itself
# to improve learning stability (otherwise the dot product produces an embedding vector with very large numbers)
USER_estimated_factors = user_profile.dot(USER_profile_factors).ravel()/user_profile.nnz
USER_estimated_factors

array([0.45832024, 0.477863  , 0.53964241, 0.47105518, 0.45585657,
       0.52164688, 0.49402138, 0.38681631, 0.43618032, 0.53817768])

In [11]:
predicted_rating = np.dot(USER_estimated_factors, ITEM_factors[item_id,:])
predicted_rating

2.668082546888111

#### The first predicted rating is a random prediction, essentially

### Step 3: We compute the prediction error and update the latent factor matrices

In [12]:
prediction_error = rating - predicted_rating
prediction_error

-1.6680825468881109

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values latent factor matrices

### Which latent factors we modify? All the factors of the item and user we used

In [13]:
# Copy original value to avoid messing up the updates
H_all = ITEM_factors.copy()
W_all = USER_profile_factors.copy()

In [14]:
H_all

array([[0.17843541, 0.35820796, 0.6182482 , ..., 0.15115074, 0.17562078,
        0.12034899],
       [0.15222427, 0.96352417, 0.44824266, ..., 0.10396651, 0.91391648,
        0.23865359],
       [0.96628695, 0.26157372, 0.58548373, ..., 0.52854501, 0.88103944,
        0.33321637],
       ...,
       [0.97229864, 0.7627337 , 0.31020948, ..., 0.23492599, 0.71148478,
        0.07988665],
       [0.54613659, 0.98855058, 0.36978015, ..., 0.82541137, 0.97344637,
        0.82363698],
       [0.36939083, 0.51889602, 0.54974387, ..., 0.40351463, 0.74113613,
        0.27692855]])

In [15]:
W_all

array([[0.18856831, 0.90217612, 0.98982947, ..., 0.94541454, 0.90624135,
        0.31908818],
       [0.44137743, 0.74073275, 0.44326409, ..., 0.40621576, 0.67268469,
        0.72549581],
       [0.8739436 , 0.98006013, 0.12750054, ..., 0.37312061, 0.47513745,
        0.33573713],
       ...,
       [0.42989664, 0.20879702, 0.45678952, ..., 0.34100331, 0.56038007,
        0.26818248],
       [0.65313656, 0.61938755, 0.90260646, ..., 0.19837236, 0.48963003,
        0.45819722],
       [0.26228609, 0.52205494, 0.70232682, ..., 0.2541192 , 0.04137471,
        0.38909485]])

#### Apply the update rule

In [16]:
learning_rate = 1e-9    # Notice the low learning rate
regularization = 1e-1   # Notice the high regularization

In [17]:
user_factors_update = prediction_error * user_profile.dot(H_all) - regularization * W_all
user_factors_update

array([[-54.69393028, -47.95400053, -52.30669633, ..., -49.34506422,
        -43.52619086, -50.20775832],
       [-54.71921119, -47.93785619, -52.25203979, ..., -49.29114434,
        -43.5028352 , -50.24839908],
       [-54.76246781, -47.96178893, -52.22046344, ..., -49.28783482,
        -43.48308047, -50.20942321],
       ...,
       [-54.71806311, -47.88466262, -52.25339234, ..., -49.28462309,
        -43.49160473, -50.20266775],
       [-54.7403871 , -47.92572167, -52.29797403, ..., -49.27036   ,
        -43.48452973, -50.22166922],
       [-54.70130206, -47.91598841, -52.27794606, ..., -49.27593468,
        -43.4397042 , -50.21475898]])

In [18]:
item_factors_update = prediction_error * user_profile.dot(W_all) - regularization * ITEM_factors
item_factors_update

array([[-46.65331867, -48.65983114, -54.97207852, ..., -39.37484917,
        -44.40023389, -54.7732478 ],
       [-46.65069756, -48.72036277, -54.95507796, ..., -39.37013075,
        -44.47406346, -54.78507826],
       [-46.73210382, -48.65016772, -54.96880207, ..., -39.4125886 ,
        -44.47077575, -54.79453454],
       ...,
       [-46.73270499, -48.70028372, -54.94127465, ..., -39.3832267 ,
        -44.45382029, -54.76920156],
       [-46.69008879, -48.72286541, -54.94723171, ..., -39.44227523,
        -44.48001645, -54.8435766 ],
       [-46.67241421, -48.67589995, -54.96522808, ..., -39.40008556,
        -44.45678542, -54.78890575]])

In [19]:
USER_profile_factors += learning_rate * user_factors_update 
ITEM_factors += learning_rate * item_factors_update

### Let's check what the new prediction for the same user-item interaction would be

In [20]:
USER_estimated_factors = user_profile.dot(USER_profile_factors).ravel()/np.sqrt(user_profile.nnz)

predicted_rating = np.dot(USER_estimated_factors, ITEM_factors[item_id,:])
predicted_rating

20.838386778004946

### We are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### Let's put all together in a training loop.

In [21]:
URM_train_coo = URM_train.tocoo()

num_factors = 10
learning_rate = 1e-9    # Notice the low learning rate
regularization = 1e-1   # Notice the high regularization

USER_profile_factors = np.random.random((n_items, num_factors))
ITEM_factors = np.random.random((n_items, num_factors))

loss = 0.0
start_time = time.time()

for sample_num in range(1000000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]
    
    # Compute prediction
    user_profile = URM_train[user_id]
    
    if user_profile.nnz == 0:
        continue 
        
    USER_estimated_factors = user_profile.dot(USER_profile_factors).ravel()/np.sqrt(user_profile.nnz)
    predicted_rating = np.dot(USER_estimated_factors, ITEM_factors[item_id,:])
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2

    if np.isnan(loss):
        break 
        
    # Copy original value to avoid messing up the updates
    H_all = ITEM_factors.copy()
    W_all = USER_profile_factors.copy()
    
    # Apply the updates
    user_factors_update = prediction_error * user_profile.dot(H_all) - regularization * W_all
    item_factors_update = prediction_error * user_profile.dot(W_all) - regularization * H_all
    
    USER_profile_factors += learning_rate * user_factors_update 
    ITEM_factors += learning_rate * item_factors_update    
    
    # Print some stats
    if (sample_num +1)% 100000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 100000 in 206.44 seconds, loss is 359.89. Samples per second 484.40
Iteration 200000 in 404.78 seconds, loss is 255.32. Samples per second 494.10
Iteration 300000 in 603.30 seconds, loss is 198.04. Samples per second 497.26
Iteration 400000 in 807.36 seconds, loss is 161.85. Samples per second 495.44
Iteration 500000 in 1009.18 seconds, loss is 136.76. Samples per second 495.45
Iteration 600000 in 1206.10 seconds, loss is 118.39. Samples per second 497.47
Iteration 700000 in 1401.56 seconds, loss is 104.41. Samples per second 499.44
Iteration 800000 in 1596.73 seconds, loss is 93.40. Samples per second 501.02
Iteration 900000 in 1791.93 seconds, loss is 84.50. Samples per second 502.25
Iteration 1000000 in 1987.08 seconds, loss is 77.17. Samples per second 503.25


### What do we see? The loss generally goes down but may oscillate a bit.
### With higher learning rates or lower regularization you may see numerical instability (i.e., the loss suddendly explodes and then becomes nan, at which point some model parameters will also become none and the model is ruined)

### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [22]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds ({:.2f} minutes, {:.2f} hours)".format(estimated_seconds, estimated_seconds/60, estimated_seconds/3600))

Estimated time with the previous training speed is 158966.67 seconds (2649.44 minutes, 44.16 hours)


### AsySVD can be very slow. Each sample requires to compute dozens (or hundreds) of dot products. Cython does not help in this case because most of the computational cost is already vectorized by numpy. Tools such as PyTorch may become useful in this case because they allow to better parallelize these operations.